# Punto Bono

<hr>
<center><strong>  Taller de Optimización y Simulaciones <br> Simulación Montecarlo <br> Puthon PuLp<br><hr> Algoritmo de Dijkstra con big M-constraint y distribución normal en sus arcos</strong></center>
<hr>
<center><strong> Hecho por Alejandro Uscátegui </strong></center>
<hr>
<center><strong> Universidad Javeriana <br> 2023 <br> Prof. Gabriel Zambrano  </strong></center>
<hr>


In [1]:
import pandas as pd
import math
import numpy as np
from pulp import *
import xlsxwriter
import random

In [2]:
# Cargar datos de entrada

## Matriz de costosxArco
matrizcsv = pd.read_csv("Matrices CSV/costosDijkstra.csv", sep = ";", encoding = 'utf-8', header = None)
costosxArco = matrizcsv.to_numpy().tolist()
costosaxArcouxiliar = matrizcsv.to_numpy().tolist()
numeroVertices = len(costosxArco)
# print(matrix)


# Vector de retrasoxnodo
vectorcsvCostosxNodo = pd.read_csv("Matrices CSV/costosNodos.csv", sep = ";", encoding = 'utf-8', header = None)
costosxNodo = vectorcsvCostosxNodo[0].to_numpy().tolist()
costosxNodoAuxiliar = vectorcsvCostosxNodo[0].to_numpy().tolist()

# print(costosxNodo)

# indicamos el nodo destino
# Si el grafo base está iniciado desde un nodo 1, para efectos del optimizador, los nodos se cuentan
# desde 0, en este sentido, hay que restarle 1 la nodo elegido como destino.
nodoDestino = 32 -1

# Los valores de entrada e

In [3]:
# Creamos el problema en pulp

prob_rmc=LpProblem('Dijkstra', LpMinimize) 

In [4]:
#creamos las variables de decision

# Partimos por indicar las variables que tiene el grafo

vertices=range(0,numeroVertices)

# Arcos es una matriz de comparación, sólo para indicar las i's y j's contenidas en el problema

arcos=[(i,j) for i in vertices for j in vertices]

# Declaro la lista de variables que tienen que ser cambiadas

var_ruta=LpVariable.dicts('ruta',(vertices,vertices), lowBound=0, cat = 'Binary')
var_retrasoxnodo=LpVariable.dicts('retxnodo',(vertices), lowBound=0, cat = 'Binary')

In [5]:
# Simulación montecarlo

N=50 # Cantidad de simulaciones
FO={}
for e in range(N):
    
    for i in vertices:
        for j in vertices:
            # Esto es para la distribución normal de cada valor de los arcos con media en la matriz
            if costosxArco[i][j] != 999:
                # A la función le entran en orden los siguientes parámetros:
                # Media, desviación, y cantidad de elementos de salida, como retorna una lista
                # y queremos trabajar con un valor y no una lista, elegimos el único elemento que genera
                # ubicándole un 0 junto a la lista.
                costosaxArcouxiliar[i][j] = np.random.normal(costosxArco[i][j],0.3,1)[0]
                
    for i in vertices:
        # Aquí la misma lógica que arriba, no hay condición if, porque quiero tomar todos los valores
            costosxNodoAuxiliar[i] = np.random.normal(costosxNodo[i],1.5,1)[0]

    # Funcion Objetivo

    prob_rmc += lpSum(var_ruta[i][j]*costosaxArcouxiliar[i][j] for (i,j) in arcos) + lpSum(var_retrasoxnodo[i]*costosxNodoAuxiliar[i] for i in vertices) + costosxNodoAuxiliar[nodoDestino]

    # Restricciones

    # origen

    prob_rmc += lpSum(var_ruta[0][j] for j in vertices) == 1

    # Destino

    prob_rmc += lpSum(var_ruta[i][nodoDestino] for i in vertices) == 1

    # Balance

    for k in vertices:
        if k>0 and k!=nodoDestino:
            prob_rmc += lpSum(var_ruta[i][k] for i in vertices) == lpSum(var_ruta[k][j] for j in vertices)
            
    # Restricción de valores válidos para retrasoxnodo con Big M

    m = 50
    for i in vertices:
        for j in vertices:
            prob_rmc +=  (m*var_retrasoxnodo[i]) >=  var_ruta[i][j] 
                

    # Resolver

    status = prob_rmc.solve()    
    print("El retardo total es: " + format(value(prob_rmc.objective)))
    
    FO[e]=value(prob_rmc.objective)

El retardo total es: 31.284363087527204


C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


El retardo total es: 25.716857107841726
El retardo total es: 24.031797481516495
El retardo total es: 23.233468665927887
El retardo total es: 22.917347855641097
El retardo total es: 24.016808972502147
El retardo total es: 17.38847630654679
El retardo total es: 24.256772036087757
El retardo total es: 29.922070699764742
El retardo total es: 22.445453403220014
El retardo total es: 22.603148311506434
El retardo total es: 29.973829883258983
El retardo total es: 29.617440734057446
El retardo total es: 26.637193456797565
El retardo total es: 26.0605344316002
El retardo total es: 23.846488328258506
El retardo total es: 26.91561002618968
El retardo total es: 27.9884374024813
El retardo total es: 27.199100394842567
El retardo total es: 28.096359357612027
El retardo total es: 27.26570155883125
El retardo total es: 39.881208043274235
El retardo total es: 26.774260637387222
El retardo total es: 26.528733719720307
El retardo total es: 25.62351791713671
El retardo total es: 24.07220952097191
El retard

In [6]:
""" ruta = {}
for v in prob_rmc.variables():
    if v.value()>0:
        print(v.name, "=", v.varValue )

print("El retardo total es: " + format(value(prob_rmc.objective))) """

' ruta = {}\nfor v in prob_rmc.variables():\n    if v.value()>0:\n        print(v.name, "=", v.varValue )\n\nprint("El retardo total es: " + format(value(prob_rmc.objective))) '

In [7]:
# Exportamos resultados a excel

writer=pd.ExcelWriter('simulacionDijkstra32.xlsx', engine='xlsxwriter')
workbook=writer.book

dataf=pd.DataFrame([[key,FO[key]] for key in FO.keys()], columns=['escenario','Retardos Totales'])

dataf.to_excel(
    writer,
    index=False,
    sheet_name= 'MonteCarlo'
)

writer.save()

C:\Users\admin\AppData\Local\Temp\ipykernel_30788\532026146.py:14: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
